### Imports

In [ ]:
import cv2
from io import BytesIO
from djitellopy import Tello
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, Image

### Detection

In [ ]:
face_cascade = cv2.CascadeClassifier('./resource/haarcascade_frontalface_default.xml')
def detect(frame) -> list:
    return face_cascade.detectMultiScale(frame, 1.3, minNeighbors=5)

### Distance

In [ ]:
def focal_distance(measured_distance: float, real_width: float, width_in_rf: float):
    return (measured_distance * width_in_rf) / real_width


def distance_finder(focal_length : float, real_object_width: float, width_in_frmae: float):
    return (focal_length * real_object_width) / width_in_frmae


def distance(detect: dict):
    return distance_finder(focal_face, width_face, detect)
    
distance_known = 50
width_face = 13
data_face = detect(cv2.imread('./resource/sample.png'))
focal_face = focal_distance(distance_known, width_face , data_face[0][2])

### Movement

In [ ]:
def movement(x: int, y: int, z: int):

    if not -90 <= x <= 90 and x != 0:
        drone.rotate_counter_clockwise(int(15)) if x < 0 else drone.rotate_clockwise(int(15)) if x > 0 else None
    
    if not -70 <= y <= 70 and y != -30:
        drone.move_up(int(30)) if y < 0 else drone.move_down(int(30)) if y > 0 else None
    
    if not 100 <= z <= 200 and z != 0:
        drone.move_forward(int(z/2)) if z > 200 else drone.move_back(100) if z < 100 else None

### Main

In [ ]:
drone = Tello()
drone.connect()
print('Battery {}%'.format(drone.get_battery()))
drone.streamon()
drone.takeoff()

stream = drone.get_frame_read()
frame_h, frame_w = 720, 960

while True:
    frame = cv2.cvtColor(stream.frame, cv2.COLOR_BGR2RGB)    
    center_x, center_y, area_z = int(frame_w/2),  int(frame_h/2), 0    
    detections = detect(frame)

    for d in detections:
        (x, y, w, h) = d
        dis = int(distance(w))
        cv2.rectangle(frame,(x, y),(x + w, y + h),(0, 0, 0), 2)
        cv2.putText(frame, str(dis), (d[0], d[1]- 20), cv2.FONT_HERSHEY_COMPLEX, .7, (0, 0, 0), 2)
        center_x, center_y, area_z = x + int(h/2),  y + int(w/2), dis
        
    
    offset_x = center_x - int(frame_w/2)
    offset_y = center_y - int(frame_h/2) - 30
    
    movement(offset_x, offset_y, area_z)

    cv2.imshow('frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break


cv2.destroyAllWindows()
drone.streamoff()
drone.land()